In [1]:
# Import packages
import cvxpy as cp
from cvxpy import Minimize, Problem, Variable, multiply
import pandas as pd
import numpy as np

# Import distance matrix from csv file
# Change this path to retrieve the distance matrix file from own device
df = pd.read_csv("C:\\Users\\20ama\\Downloads\\Distance Matrix.csv")

# Convert data frame into an array
a = df.to_numpy()

# remove an unnecessary name column from matrix
dist_matrix = a[:,1:]


# Alternative: paste distance matrix from file into a matrix
dist_matrix = np.array([[0, 220, 210, 250, 89, 180, 450, 200, 500, 500, 600, 800, 550, 550, 900, 850, 450, 400, 850, 1300, 450],
                        [220, 0, 210, 260, 300, 400, 700, 260, 550, 500, 650,
                            800, 550, 500, 850, 800, 350, 180, 900, 1300, 260],
                        [210, 210, 0, 110, 210, 260, 550, 110, 400, 350, 500,
                         600, 350, 350, 700, 650, 280, 280, 750, 1100, 450],
                        [250, 260, 110, 0, 220, 180, 450, 10, 300, 270, 400,
                         550, 350, 400, 700, 650, 260, 350, 650, 1100, 500],
                        [89, 300, 210, 220, 0, 95, 400, 110, 400, 400, 500,
                            700, 500, 600, 850, 850, 450, 450, 750, 1300, 550],
                        [180, 400, 260, 180, 95, 0, 300, 67, 350, 350, 450,
                            650, 450, 550, 800, 800, 450, 500, 700, 1200, 600],
                        [450, 700, 550, 450, 400, 300, 0, 270, 350, 500, 500,
                         700, 650, 800, 900, 1000, 750, 800, 700, 1300, 900],
                        [200, 260, 110, 10, 110, 67, 270, 0, 200, 240, 350,
                         500, 260, 260, 600, 500, 140, 250, 650, 1000, 450],
                        [500, 550, 400, 300, 400, 350, 350, 200, 0, 180, 160,
                         350, 300, 500, 550, 650, 500, 650, 400, 1000, 800],
                        [500, 500, 350, 270, 400, 350, 500, 240, 180, 0, 170,
                         350, 180, 350, 500, 550, 400, 500, 450, 900, 700],
                        [600, 650, 500, 400, 500, 450, 500, 350, 160, 170, 0,
                         200, 280, 450, 550, 600, 550, 650, 270, 950, 850],
                        [800, 800, 600, 550, 700, 650, 700, 500, 350, 350, 200,
                         0, 300, 400, 500, 550, 600, 700, 350, 900, 1000],
                        [550, 550, 350, 350, 500, 450, 650, 260, 300, 180, 280,
                         300, 0, 170, 350, 450, 300, 400, 550, 800, 700],
                        [550, 500, 350, 400, 600, 550, 800, 260, 500, 350, 450,
                         400, 170, 0, 400, 270, 200, 400, 700, 750, 650],
                        [900, 850, 700, 700, 850, 800, 900, 600, 550, 500, 550,
                         500, 350, 400, 0, 130, 550, 750, 700, 140, 1100],
                        [850, 800, 650, 650, 850, 800, 1000, 500, 650, 550, 600,
                         550, 450, 270, 130, 0, 450, 600, 850, 500, 1000],
                        [450, 350, 280, 260, 450, 450, 750, 140, 500, 400, 550,
                         600, 300, 200, 550, 450, 0, 180, 800, 900, 500],
                        [400, 180, 280, 350, 450, 500, 800, 250, 650, 500, 650,
                         700, 400, 400, 750, 600, 180, 0, 950, 1100, 350],
                        [850, 900, 750, 650, 750, 700, 700, 650, 400, 450, 270,
                         350, 550, 700, 700, 850, 800, 950, 0, 1100, 1100],
                        [1300, 1300, 1100, 1100, 1300, 1200, 1300, 1000, 1000, 900,
                         950, 900, 800, 750, 140, 500, 900, 1100, 1100, 0, 1400],
                        [450, 260, 450, 500, 550, 600, 900, 450, 800, 700, 850, 1000, 700, 650, 1100, 1000, 500, 350, 1100, 1400, 0]])

# Create indexes
locations = 21
no_dining = 18

# Create decision variables
x = cp.Variable((locations, locations), boolean=True)
t = cp.Variable(locations, nonneg=True)

# Objective function
obj_func = cp.sum(cp.multiply(dist_matrix, x))

# Create constraints

# Every arrival to a location has to have a departure from a location
constraints = []
for j in range(no_dining):
    constraints.append(cp.sum(x[:, j]) == 1)

# Every departure from a location has to have an arrival to a location
for i in range(no_dining):
    constraints.append(cp.sum(x[i, :]) == 1)

# No subtours
for i in range(1,locations):
    for j in range(1,locations):
        constraints.append(t[i]-t[j]+locations*x[i, j] <= locations-1)

# No self loops (each diagonal element in the distance matrix is zero)
for i in range(locations):
    constraints.append(x[i, i] == 0) 

# No consecutive libraries
constraints.append(x[10, 13] == 0)
constraints.append(x[13, 10] == 0)

# Restricted time places are visited last
constraints.append(x[2, 3] == 1)
constraints.append(x[3, 4] == 1)
constraints.append(x[4, 5] == 1)
constraints.append(x[5, 0] == 1)

# Only one lunch location visited
constraints.append(cp.sum(x[18, :]+x[19, :] + x[20, :]) == 1)
constraints.append(cp.sum(x[:, 18]+x[:, 19]+x[:, 20]) == 1)

# a dining hall cannot arrive or depart from another dining hall
for i in range(18,21):
    for j in range(18,21):
        constraints.append(x[i,j] == 0)

# an arrival to a location i must have a departure from location j (a lunch loaction)
# no arrival to a location i will not have a departure from location j (a lunch location)
for j in range(18,21):
    constraints.append(cp.sum(x[0:18,j])==cp.sum(x[j,0:18]))

# Solve problem
problem = cp.Problem(cp.Minimize(obj_func), constraints)
problem.solve(solver=cp.GUROBI, verbose=True)

# Print function and values
print("obj_func = ")
print(obj_func.value)
print("x = ")
print(x.value)


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Dec 05 09:51:33 AM: Your problem has 462 variables, 477 constraints, and 0 parameters.
(CVXPY) Dec 05 09:51:33 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 05 09:51:33 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 05 09:51:33 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 05 09:51:33 AM: Compiling problem (target solver=GUROBI).
(CVXPY) Dec 05 09:51:33 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStu